In [1]:
JsonStruct = {
  "data": {
    "type": "",
    "attributes": {
      "name": "",
      "affected_organ": "",
      "age": "",
      "diagnostic_main": "",
      "gender": "",
      "medical_history": ""
    }
  }
}

In [2]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument as pdf1

In [3]:
fileName = "1-s2.0-S1138359320302823-main.pdf"

In [4]:
fd = open(fileName, "rb")
parser = PDFParser(fd)
doc = pdf1(parser)

In [5]:
doc.info[0]

{'Author': b'A. Alc\xe1ntara Montero',
 'CreationDate': b'D:20200918123156Z',
 'Creator': b'Elsevier',
 'CrossMarkDomains[1]': b'sciencedirect.com',
 'CrossMarkDomains[2]': b'elsevier.com',
 'CrossmarkDomainExclusive': b'true',
 'CrossmarkMajorVersionDate': b'2010-04-23',
 'ElsevierWebPDFSpecifications': b'7.0',
 'Keywords': b'',
 'ModDate': b"D:20200918125129+05'30'",
 'Producer': b'Acrobat Distiller 9.0.0 (Windows)',
 'Subject': b'Medicina de Familia. SEMERGEN, 46 (2020) 365-367. doi:10.1016/j.semerg.2020.08.002',
 'Title': b'COVID-19 y dolor cr\xf3nico: muchos interrogantes y pocas certezas',
 'Trapped': /'True',
 'doi': b'10.1016/j.semerg.2020.08.002',
 'robots': b'noindex'}

In [6]:
JsonStruct["data"]["attributes"]["diagnostic_main"] = doc.info[0]['Title'].decode("windows-1252")
JsonStruct["data"]["attributes"]["medical_history"] = doc.info[0]['Keywords'].decode("windows-1252") 

JsonStruct

{'data': {'type': '',
  'attributes': {'name': '',
   'affected_organ': '',
   'age': '',
   'diagnostic_main': 'COVID-19 y dolor crónico: muchos interrogantes y pocas certezas',
   'gender': '',
   'medical_history': ''}}}

In [7]:
import pdfreader
from pdfreader import PDFDocument, SimplePDFViewer

In [8]:
fd = open(fileName, "rb")
doc = PDFDocument(fd)

In [9]:
all_pages = [p for p in doc.pages()]
# Total Pages
totalPages = len(all_pages)

totalPages

3

In [10]:
viewer = SimplePDFViewer(fd)

pdfContent = ""

for x in range(totalPages):
  viewer.navigate(x+1)
  viewer.render()
  pdfContent += ' '.join(viewer.canvas.strings)
    
pdfContent = pdfContent.replace('"', "")

pdfContent

'Semergen.   2020; 46(6) :365- - -367 www .elsevier .es/semergen EDIT ORIAL COVID-19   y   dolor   crónico:   muchos   interrogantes y   pocas   certezas COVID-19   and   chronic   pain:   Many   questions   and   few   certainties En   diciembre   del   2019,   la   Comisión   Municipal   de   Salud   y   Sani- dad   de   W uhan   (provincia   de   Hubei,   China)   informó   de   una serie   de   casos   de   neumonía   de   etiología   desconocida,   con una   exposición   común   en   un   mercado   mayorista   de   marisco, pescado   y   animales   vivos   en   la   ciudad   de   W uhan.   El   7   de enero   del   2020,   las   autoridades   chinas   identiﬁcaron   como agente   causante   del   brote   un   nuevo   tipo   de   virus   de   la   fami- lia   Coronaviridae ,   denominado   SARS-CoV -2,   que   produce la   enfermedad   COVID-19   (acrónimo   del   inglés   coronavirus disease   2019 ).   Desde   entonces   se   han   notiﬁcado   miles   de casos   con   una   disem

In [11]:
JsonStruct["data"]["attributes"]["medical_history"] += pdfContent

JsonStruct

{'data': {'type': '',
  'attributes': {'name': '',
   'affected_organ': '',
   'age': '',
   'diagnostic_main': 'COVID-19 y dolor crónico: muchos interrogantes y pocas certezas',
   'gender': '',
   'medical_history': 'Semergen.   2020; 46(6) :365- - -367 www .elsevier .es/semergen EDIT ORIAL COVID-19   y   dolor   crónico:   muchos   interrogantes y   pocas   certezas COVID-19   and   chronic   pain:   Many   questions   and   few   certainties En   diciembre   del   2019,   la   Comisión   Municipal   de   Salud   y   Sani- dad   de   W uhan   (provincia   de   Hubei,   China)   informó   de   una serie   de   casos   de   neumonía   de   etiología   desconocida,   con una   exposición   común   en   un   mercado   mayorista   de   marisco, pescado   y   animales   vivos   en   la   ciudad   de   W uhan.   El   7   de enero   del   2020,   las   autoridades   chinas   identiﬁcaron   como agente   causante   del   brote   un   nuevo   tipo   de   virus   de   la   fami- lia   Coronavi

## Persist Json

In [12]:
fileRaw = fileName.split(".")

fileNameJson = "".join(fileRaw[:-1]) + ".json"

fileNameJson

'1-s20-S1138359320302823-main.json'

In [13]:
import json

with open(fileNameJson, 'w', encoding='utf8') as json_file:
  json.dump(JsonStruct, json_file, ensure_ascii=False)

## To calculate Keywords

In [14]:
import pandas as pd

text = viewer.canvas.strings

df = pd.DataFrame(text,columns=['keywords'])

grouped = df.groupby('keywords').size().reset_index(name='counts').sort_values('counts',ascending=False)
grouped


,keywords,counts
0,,430
209,de,25
270,la,13
394,y,9
335,que,8
...,...,...
179,atentos,1
44,Agency,1
177,asistenciales,1
176,asconcelos,1


In [15]:
# Term Frequency
# TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document).

total_terms = grouped["keywords"].count()

grouped['tf'] = grouped['counts'].apply(lambda x: x/total_terms)
grouped.sort_values('tf',ascending=False)

,keywords,counts,tf
0,,430,1.0750
209,de,25,0.0625
270,la,13,0.0325
394,y,9,0.0225
335,que,8,0.0200
...,...,...,...
396,»,1,0.0025
397,"éticas,",1,0.0025
2,(AINS),1,0.0025
7,-23.,1,0.0025


In [16]:
grouped[grouped['keywords'].str.len() >= 3].head(20)


,keywords,counts,tf
335,que,8,0.0200
313,para,5,0.0125
274,los,5,0.0125
223,dolor,5,0.0125
187,chronic,4,0.0100
68,"Cáceres,",4,0.0100
194,con,4,0.0100
172,and,3,0.0075
257,https://www,3,0.0075
203,crónico,3,0.0075


In [17]:
grouped[grouped['tf'] <= 0.01].head(20)


,keywords,counts,tf
187,chronic,4,0.0100
68,"Cáceres,",4,0.0100
398,˜,4,0.0100
194,con,4,0.0100
3,",",4,0.0100
130,R,4,0.0100
95,J,4,0.0100
158,a,4,0.0100
172,and,3,0.0075
257,https://www,3,0.0075
